<a href="https://colab.research.google.com/github/LEECHANGDAEE/colab/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%ACnew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, get_scorer
from sklearn.linear_model import Lasso, Ridge, LassoCV,LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split
from numpy import arange

In [202]:
train = pd.read_csv('./train2.csv',encoding='cp949')
test = pd.read_csv('./test1.csv',encoding='cp949')
submission = pd.read_csv('./sample_submission1.csv',encoding='cp949')

train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

train= pd.get_dummies(data = train, columns = ['임대건물구분'], prefix = '건물구분')
test =  pd.get_dummies(data = test, columns = ['임대건물구분'], prefix = '건물구분')


In [203]:
# for i in range(len(train)) :
#   if train.자격유형[i] in ['C','F'] :
#     train.자격유형[i] = 'C'
#   elif train.자격유형[i] in ['H','B','O'] :
#     train.자격유형[i] = 'H'
# for i in range(len(test)) :
#   if test.자격유형[i] in ['C','F'] :
#     test.자격유형[i] = 'C'
#   elif test.자격유형[i] in ['H','B','O'] :
#     test.자격유형[i] = 'H'
# train= pd.get_dummies(data = train, columns = ['자격유형'], prefix = '유형')
# test =  pd.get_dummies(data = test, columns = ['자격유형'], prefix = '유형')

In [204]:
data_m = train.groupby('단지코드').sum()
data_mt = test.groupby('단지코드').sum()
# data_n = data_m.유형_N
# data_nt = data_mt.유형_N

In [205]:


len(data_m)
data_st = data_m.건물구분_상가
data_ap = data_m.건물구분_아파트
# data_a = data_m.유형_A
# data_c = data_m.유형_C
# data_d = data_m.유형_D
# data_e = data_m.유형_E
# data_g = data_m.유형_G
# data_h = data_m.유형_H
# data_i = data_m.유형_I
# data_j = data_m.유형_J
# data_k = data_m.유형_K
# data_l = data_m.유형_L
# data_m = data_m.유형_M

data_stt = data_mt.건물구분_상가
data_apt = data_mt.건물구분_아파트
# data_at = data_mt.유형_A
# data_ct = data_mt.유형_C
# data_dt = data_mt.유형_D
# data_et = data_mt.유형_E
# data_gt = data_mt.유형_G
# data_ht = data_mt.유형_H
# data_it = data_mt.유형_I
# data_jt = data_mt.유형_J
# data_kt = data_mt.유형_K
# data_lt = data_mt.유형_L
# data_mt = data_mt.유형_M


In [206]:
# print(train.공급유형.value_counts())
# print(test.공급유형.value_counts())


In [207]:
for i in range(len(train)) :
  if train.공급유형[i] in ['임대상가','영구임대'] :
    train.공급유형[i] = '영구임대_상가'
  elif train.공급유형[i] in ['공공임대(10년)','공공임대(50년)','공공임대(분납)','공공분양','공공임대(5년)'] :
    train.공급유형[i] = '공공임대'
  elif train.공급유형[i] in ['장기전세','행복주택'] :
    train.공급유형[i] = '전세주택'
#train.공급유형.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [208]:
for i in range(len(test)) :
  if test.공급유형[i] in ['임대상가','영구임대'] :
    test.공급유형[i] = '영구임대_상가'
  elif test.공급유형[i] in ['공공임대(10년)','공공임대(50년)','공공임대(분납)','공공분양','공공임대(5년)'] :
    test.공급유형[i] = '공공임대'
  elif test.공급유형[i] in ['장기전세','행복주택'] :
    test.공급유형[i] = '전세주택'

#test.공급유형.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [209]:
train['전용면적'] = round(train['전용면적'])//5*5
test['전용면적'] = round(test['전용면적'])//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 1
idx = train[train['전용면적']>50].index
train.loc[idx, '전용면적'] = 2
idx = train[train['전용면적']==50].index
train.loc[idx, '전용면적'] = 3
idx = train[train['전용면적']>40].index
train.loc[idx, '전용면적'] = 4
idx = train[train['전용면적']>9].index
train.loc[idx, '전용면적'] = 5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 1
idx = test[test['전용면적']>50].index
test.loc[idx, '전용면적'] = 2
idx = test[test['전용면적']==50].index
test.loc[idx, '전용면적'] = 3
idx = test[test['전용면적']>40].index
test.loc[idx, '전용면적'] = 4
idx = test[test['전용면적']>9].index
test.loc[idx, '전용면적'] = 5



columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스','공급유형']
target = '등록차량수'
area_columns = []

for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

new_train = pd.DataFrame()
new_test = pd.DataFrame()

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
  
new_train = new_train.fillna(0)
new_test = new_test.fillna(0)


411it [00:04, 90.60it/s]
150it [00:01, 95.49it/s]


In [210]:

nw = new_train.set_index('단지코드')
ntw = new_test.set_index('단지코드')


In [211]:
# ,data_a,data_c,data_d,data_e,data_g,data_h,data_i,data_j,data_k,data_l,data_m,data_n
# ,data_at,data_ct,data_dt,data_et,data_gt,data_ht,data_it,data_jt,data_kt,data_lt,data_mt,data_nt

In [212]:
d1 = pd.concat([nw.iloc[:,:-1],data_ap,data_st],axis=1)
#d2 = pd.concat([d1,data_st],axis=1)
d1 = d1.reset_index()

dt1 = pd.concat([ntw,data_apt,data_stt],axis=1)
#d2 = pd.concat([d1,data_st],axis=1)
dt1 = dt1.reset_index()

error = ['C2085', 'C1397','C2431', 'C1649','C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
idx = []
for i in range(len(error)):
  idx = np.append(idx,d1[d1['index']==error[i]].index)
d1.drop(idx,inplace=True)
idx = []
for i in range(len(error)):
  idx = np.append(idx,dt1[dt1['index']==error[i]].index)
dt1.drop(idx,inplace=True)



In [213]:
print(dt1.shape)
print(d1.shape)


(150, 15)
(411, 15)


In [214]:

x = d1.iloc[:,1:]
y = new_train['등록차량수']

In [215]:
yy = dt1.iloc[:,1:]

In [216]:
for i in range(len(x)) : 
  if x.지역[i] in ['서울특별시','부산광역시'] :
    x.지역[i] = '경부'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [217]:
for i in range(len(yy)) : 
  if yy.지역[i] in ['서울특별시','부산광역시'] :
    yy.지역[i] = '경부'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [218]:
# for i in range(len(x)) :
#      if x.지역[i] in ['광주광역시','부산광역시','대전광역시','울산광역시','대구광역시'] :
#     x.지역[i] = '광역시'
#   elif x.지역[i] in ['서울특별시','세종특별자치시','제주특별자치도'] :
#     x.지역[i] = '특별자치시'
#   elif x.지역[i] in ['경상남도','경상북도','충청북도','충청남도','전라남도','전라북도'] :
#     x.지역[i] = '지방시'

In [219]:
# for i in range(len(yy)) :
#   if yy.지역[i] in ['광주광역시','부산광역시','대전광역시','울산광역시','대구광역시'] :
#     yy.지역[i] = '광역시'
#   elif yy.지역[i] in ['서울특별시','세종특별자치시','제주특별자치도'] :
#     yy.지역[i] = '특별자치시'
#   elif yy.지역[i] in ['경상남도','경상북도','충청북도','충청남도','전라남도','전라북도'] :
#     yy.지역[i] = '지방시'

In [220]:
x = pd.get_dummies(data = x, columns = ['지역'], prefix = '지역')
x = pd.get_dummies(data = x, columns = ['공급유형'], prefix = '공급유형')
yy = pd.get_dummies(data = yy, columns = ['지역'], prefix = '지역')
yy = pd.get_dummies(data = yy, columns = ['공급유형'], prefix = '공급유형')

In [221]:
x = x.drop('건물구분_아파트',axis=1)

In [222]:
x.총세대수 = np.sqrt(x.총세대수)
x.단지내주차면수 = np.sqrt(x.단지내주차면수)

In [223]:
y = np.sqrt(y)

In [224]:
#  x =x.iloc[:,1:]
#  yy = yy.iloc[:,1:]


In [225]:
# for i in range(50) :
  
#   x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =i)
#   mlr = LinearRegression()
#   mlr.fit(x_train, y_train)

#   ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

#   lasso = Lasso(alpha=0.2).fit(x_train, y_train)

#   y_linear_predict = mlr.predict(x_test)
#   y_ridge_predict = ridge.predict(x_test)
#   y_lasso_predict = lasso.predict(x_test)
#   from sklearn.metrics import mean_absolute_error
#   from sklearn.metrics import mean_squared_error
#   print(mean_absolute_error(y_lasso_predict,y_test))

In [226]:

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =0)
mlr = LinearRegression()
mlr.fit(x_train, y_train)

ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

lasso = Lasso(alpha=0.23).fit(x_train, y_train)

y_linear_predict = mlr.predict(x_test)
y_ridge_predict = ridge.predict(x_test)
y_lasso_predict = lasso.predict(x_test)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('linear regression rmse : {}'.format(mean_squared_error(y_linear_predict,y_test)**0.5))
print('ridge regression rmse : {}'.format(mean_squared_error(y_ridge_predict,y_test)**0.5))
print('lasso regression rmse : {}'.format(mean_squared_error(y_lasso_predict,y_test)**0.5))
print(mean_absolute_error(lasso.predict(x_train),y_train))
print(mean_absolute_error(y_lasso_predict,y_test))


linear regression rmse : 2.6459011703962614
ridge regression rmse : 2.6298499647321374
lasso regression rmse : 2.625748761624344
2.4804992026552712
1.9561209887278386


In [227]:
yy = yy.drop('건물구분_아파트',axis=1)
yy.총세대수 = np.sqrt(yy.총세대수)
yy.단지내주차면수 = np.sqrt(yy.단지내주차면수)

In [228]:
# y_lasso_predict = lasso.predict(yy)
# submission['num'] = y_lasso_predict

# submission.to_csv('lasso.csv', index=False)

In [229]:
random_grid = {'alpha':  [round(x,1) for x in arange(0, 1, 0.01)]}

In [230]:
from sklearn.model_selection import RandomizedSearchCV,cross_val_score, KFold
kfold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

lasso = Lasso()
# lasso_random = RandomizedSearchCV(estimator = lasso, param_distributions = random_grid, n_iter = 100, cv = kfold, verbose=2, n_jobs = -1)
lasso_random = GridSearchCV(estimator= lasso, param_grid= random_grid, scoring='neg_mean_absolute_error', cv=kfold, n_jobs=-1)

In [231]:
lasso_random_cv = lasso_random.fit(x_train,y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1599.7324323470316, tolerance: 2.064496754339715
  positive)


In [232]:
lasso_random_cv.best_estimator_

Lasso(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [233]:
lasso_optimal = lasso_random_cv.best_estimator_.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1599.7324323470316, tolerance: 2.064496754339715
  positive)


In [234]:
la_op_pred = lasso_optimal.predict(x_test)
print('lasso regression rmse : {}'.format(mean_squared_error(la_op_pred,y_test)**0.5))
print(mean_absolute_error(la_op_pred,y_test))

lasso regression rmse : 2.645901170396538
2.045312246888078


In [ ]:
scores = cross_val_score(lasso_optimal, x_train, y_train, cv=kfold,scoring='neg_mean_absolute_error')
print(np.mean(scores))

In [245]:
y_lasso_predict = lasso_optimal.predict(yy)
y_lasso_predict = y_lasso_predict**2
submission['num'] = y_lasso_predict

submission.to_csv('lasso.csv', index=False)

In [246]:
print('lasso regression rmse : {}'.format(mean_squared_error(lasso_optimal.predict(x_test),y_test)**0.5))
print(mean_absolute_error(lasso_optimal.predict(x_train),y_train))
print(mean_absolute_error(lasso_optimal.predict(x_test),y_test))

lasso regression rmse : 2.645901170396538
2.2374464929519817
2.045312246888078
